<a href="https://colab.research.google.com/github/ankile/MIT6.036-Intro-Machine-Learning/blob/master/hw-08/ankile_MIT6_036_Hw_08_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MIT 6.036 Fall 2020: Homework 8#

This colab notebook provides code and a framework for [homework 8](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week8/week8_homework/).  You can work out your solutions here, then submit your results back on the homework page when ready.

## <section>**Setup**</section>

First, download the code distribution for this homework that contains test cases and helper functions.

Run the next code block to download and import the code for this lab.


In [0]:
!rm -rf code_for_hw8*
!rm -rf data
!rm -rf mnist_data
!rm -rf *.zip
!rm -rf test*/
!rm -rf *.py
!rm -rf *.pt
!rm -rf __*
!wget --quiet https://introml.odl.mit.edu/cat-soop/_static/6.036/homework/hw08/code_for_hw8.zip
!unzip code_for_hw8.zip
!unzip code_for_hw8/q4.zip
!unzip -q test1.zip
!unzip -q test2.zip
!unzip -q test3.zip
!mv code_for_hw8/* .

import numpy as np
import itertools

import math as m

import torch
from torch.nn import (Linear, ReLU, Conv1d, Flatten, Conv2d, Sequential, 
                      MaxPool1d, MaxPool2d, Dropout, CrossEntropyLoss, MSELoss)
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from matplotlib import pyplot as plt

from torchvision.datasets import MNIST
import torchvision

from skimage.io import imread, imshow
from skimage.transform import resize

import os

from code_for_hw8_pytorch import get_image_data_1d

from utils_hw8 import (model_fit, model_evaluate, run_pytorch, call_model, 
                       plot_decision, plot_heat, plot_separator, make_iter, 
                       set_weights, set_bias)


Archive:  code_for_hw8.zip
   creating: code_for_hw8/
  inflating: code_for_hw8/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/code_for_hw8/
  inflating: __MACOSX/code_for_hw8/._.DS_Store  
  inflating: code_for_hw8/q4.zip     
  inflating: code_for_hw8/utils_hw8.py  
  inflating: code_for_hw8/code_for_hw8_oop.py  
  inflating: code_for_hw8/code_for_hw8_pytorch.py  
   creating: code_for_hw8/data/
  inflating: code_for_hw8/data/data3_train.csv  
  inflating: code_for_hw8/data/data4_train.csv  
  inflating: code_for_hw8/data/data4_validate.csv  
  inflating: code_for_hw8/data/data3_validate.csv  
  inflating: code_for_hw8/data/dataXor_train.csv  
  inflating: code_for_hw8/data/data2_train.csv  
  inflating: code_for_hw8/data/data2_validate.csv  
  inflating: code_for_hw8/data/data3class_train.csv  
  inflating: code_for_hw8/data/data1_validate.csv  
  inflating: code_for_hw8/data/data1_train.csv  
Archive:  code_for_hw8/q4.zip
  inflating: test3.zip               
  inflating:

# 1) Implementing Mini-batch Gradient Descent and Batch Normalization

** Note: You can click the arrow on the left of this text block to collapse/expand this optional section and all its code blocks **

Last week we implemented a framework for building neural networks from scratch. We trained our models using *stochastic* gradient descent. In this problem, we explore how we can implement batch normalization as a module `BatchNorm` in our framework. It is the same module which you analyzed in problem 1. 

Key to the concept of batch normalization is the doing gradient descent on batches of data. So we instead of using last week's stochastic gradient descent, we will first implement the *mini-batch* gradient descent method `mini_gd`, which is a hybrid between *stochastic* gradient descent and *batch* gradient descent. The lecture notes on <a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week7/neural_networks_2/1?activate_block_id=block-v1%3AMITx%2B6.036%2B2019_Spring%2Btype%40vertical%2Bblock%40neural_networks_2_optimizing_neural_network_parameters_vert"> optimizing neural network parameters</a> are helpful for this part.

In *mini-batch* gradient descent, for a mini-batch of size $K$, we select $K$ distinct data points uniformly at random from the data set and update the network weights based only on their contributions to the gradient:
$$W := W - \eta\sum_{i=1}^K \nabla_W \mathcal{L}(h(x^{(i)}; W), y^{(i)})\;\;.$$

Our *mini-batch* method `mini_gd` will be implemented within the `Sequential` python class (see homework 7 problem 2) and will take the following as inputs:

* `X`: a standard data array (d by n)
* `y`: a standard labels row vector (1 by n)
* `iters`: the number of updates to perform on weights $W$
* `lrate`: the learning rate used
* `K`: the mini-batch size to be used

One call of `mini_gd` should call `Sequential.backward` for back-propagation and `Sequential.step` for updating the weights, for a total of `iters` times, using `lrate` as the learning rate. As in our implementation of `sgd` from homework 7, we compute the predicted output for a mini-batch of data with the `Sequential.forward` method. We compute the loss between our predictions and the true labels using the assigned `Sequential.loss` method. (Note that in homework 7, `Sequential.step` was called `Sequential.sgd_step`. While the functionality of the step function is the same, it has been renamed for convenience. The same is true for the `module.step` function of each module we implemented, where applicable.)

For picking $K$ unique data points at random from our large data-set for each mini-batch, we will implement the following strategy: we will first shuffle our data points `X` (and associated labels `y`). Then, we get $\frac{n}{k}$ (rounded down to the nearest integer) different mini-batches by grouping each $K$ consecutive points from this shuffled array. If we end up iterating over all the points but need more mini-batches, we will repeat the shuffling and the batching process.

<b>1A)</b> You need to fill in the missing code below. We have implemented the shuffling of indices and have provided you with the outer and inner loops.

### PLEASE COPY IN YOUR CODE FROM HOMEWORK 7 TO COMPLEMENT THE CLASSES GIVEN HERE

Recall that your implementation from homework 7 included the following classes:
    
  * Module
  * Linear 
  * Tanh 
  * ReLU 
  * SoftMax
  * NLL
  * Sequential

In [0]:
import math as m
class Module:
    def sgd_step(self, lrate): pass  # For modules w/o weights


class Linear(Module):
    def __init__(self, m, n):
        self.m, self.n = (m, n)  # (in size, out size)
        self.W = np.random.normal(0, 1.0 * m ** (-.5), [m, n])  # (m x n)
        self.W0 = np.zeros([self.n, 1])  # (n x 1)

    def forward(self, A):
        # (m x b)  Hint: make sure you understand what b stands for
        self.A = A
        return self.W.T@A + self.W0  # (n x b)

    # dLdZ is (n x b), uses stored self.A
    def backward(self, dLdZ):
        self.dLdW = self.A@dLdZ.T
        self.dLdW0 = np.sum(dLdZ, axis=1, keepdims=True)
        return self.W@dLdZ           # Return dLdA (m x b)

    def sgd_step(self, lrate):  # Gradient descent step
        self.W -= lrate * self.dLdW
        self.W0 -= lrate * self.dLdW0

class Tanh(Module):            # Layer activation
    def forward(self, Z):
        self.A = np.tanh(Z)
        return self.A

    # Uses stored self.A
    def backward(self, dLdA):
        # Your code: return dLdZ with dimensions (n, b) (?, b)
        return dLdA * (1 - self.A**2)


class ReLU(Module):              # Layer activation
    def forward(self, Z):
        self.A = np.maximum(Z, np.zeros(Z.shape))             # Your code: (n, b)
        return self.A

    # uses stored self.A
    def backward(self, dLdA):
        # Your code: return dLdZ (m, b)
        return dLdA * (self.A != 0)


class SoftMax(Module):           # Output activation
    def forward(self, Z):
        # Your code: (n, b)
        return np.apply_along_axis(lambda col: np.exp(col) / np.sum(np.exp(col)), axis=0, arr=Z)              # Your code: (n, b)

    # Assume that dLdZ is passed in
    def backward(self, dLdZ):
        return dLdZ

    def class_fun(self, Ypred):  # Return class indices
        return np.argmax(Ypred, axis=0)              # Your code: A 1D vector (b, ) 


class NLL(Module):       # Loss
    def forward(self, Ypred, Y):
        self.Ypred = Ypred
        self.Y = Y
        return -np.sum(Y * np.log(Ypred))      # Your code

    def backward(self):  # Use stored self.Ypred, self.Y
        return self.Ypred - self.Y      # Your code

Implement `mini_gd` in `Sequential` below.

**Hint:** The documentation for <a href="https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.shuffle.html"> `numpy.random.shuffle`</a> might be helpful for this part. If you have a list of elements `l` and a set of indices `indices`, you can call `numpy.random.shuffle(indices)` and `l = l[indices]` to shuffle the elements of `l`.

In [0]:
import math as m
class Sequential:
    def __init__(self, modules, loss):            
        self.modules = modules
        self.loss = loss

    def mini_gd(self, X, Y, iters, lrate, notif_each=None, K=10):
        D, N = X.shape

        np.random.seed(0)
        num_updates = 0
        indices = np.arange(N)
        while num_updates < iters:

            np.random.shuffle(indices)
            X = X[:, indices]
            Y = Y[:, indices]

            for j in range(m.floor(N/K)):
                if num_updates >= iters: break

                # Implement the main part of mini_gd here
                Xt = X[:, j*K:(j+1)*K]
                Yt = Y[:, j*K:(j+1)*K]

                # The rest of this function should be similar to your
                # implementation of Sequential.sgd in HW 7
                Ypred = self.forward(Xt)

                loss = self.loss.forward(Ypred, Yt)

                delta = self.loss.backward()
                self.backward(delta)
                self.sgd_step(lrate)
                
                num_updates += 1

    def forward(self, Xt):                        
        for m in self.modules: Xt = m.forward(Xt)
        return Xt

    def backward(self, delta):                   
        for m in self.modules[::-1]: delta = m.backward(delta)

    def sgd_step(self, lrate):    
        for m in self.modules: m.sgd_step(lrate)

<b>1B)</b> We are now ready to implement batch normalization into our neural network framework! Our module `BatchNorm` will sit between consecutive layers of neurons, such as the $l^{th}$ and $(l+1)^{th}$ layers, acting as a "corrector" which allows $W^l$ to change freely, producing outputs $z^l$, but then the module corrects the covariate shift induced in the signals before they reach the $(l+1)^{th}$ layer, converting $z^l$ to $\widehat{Z}^l$. 

The following is a summmary what is described in the <a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week7/neural_networks_2/2">lecture notes</a>, and it should guide your implementation of the module. 

Any normalization between the $l^{th}$ and $(l+1)^{th}$ layers is done *separately* for each of the $n^l$ input connections leading to the $(l+1)^{th}$ layer. We handle a mini-batch of data of size $K$, and $Z^l$ is $n^l \times K$, and the output $\widehat{Z}^l$is of the same shape. 

We first compute $n^l$ *batchwise* means and
standard deviations.  Let $\mu^l$ be the $n^l \times 1$ vector (`self.mus`) where
$$\mu^l_i = \frac{1}{K} \sum_{j = 1}^K Z^l_{ij}\;\;,$$
and let $\sigma^l$ be the $n^l \times 1$ vector (`self.vars`) where 
$$\sigma^l_i = \sqrt{\frac{1}{K} \sum_{j = 1}^K (Z^l_{ij} - \mu_i)^2}\;\;.$$

The normalized data `self.norm` is the matrix $\overline{Z}$, where
$$\overline{Z}^l_{ij} = \frac{Z^l_{ij} - \mu^l_i}{\sigma^l_i + \epsilon}\;\;,$$
and where $\epsilon$ is a very small constant to guard against division by
zero. 

We define weights $G^l$ (`self.G`) and $B^l$ (`self.B`), each being an $n^l \times 1$ vector, which we use to to shift and scale the outputs:
$$\widehat{Z}^l_{ij} = G^l_i \overline{Z}^l_{ij} + B^l_i\;\;.$$

The outputs are finally ready to be passed to the $(l+1)^{th}$ layer.

A slight warning (that we will not worry about here) about `BatchNorm` is that during the *test* phase, if the test mini-batch size is too small (imagine we are deploying a neural network that deals with live video frames), then the lack of samples would cause the freshly-calculated $\mu^l$ and $\sigma^l$ to be far off from their true values that the module's parameters $G^l$ and $B^l$ were trained to be compatible with. To fix that, people usually compute a running average of $\mu^l$ and $\sigma^l$ during training, to be used at test time. We will assume our test mini-batches are large enough.

In this problem we only implement the `BatchNorm.forward` and `BatchNorm.step` methods. We provide you with the implementation for `BatchNorm.backward` and the lecture notes contain the details of the derivations. You will need to fill in the missing code below.

In [0]:
class BatchNorm(Module):    
    def __init__(self, m):
        np.random.seed(0)
        self.eps = 1e-20
        self.m = m  # number of input channels
        
        # Init learned shifts and scaling factors
        self.B = np.zeros([self.m, 1]) # m x 1
        self.G = np.random.normal(0, 1.0 * self.m ** (-.5), [self.m, 1]) # m x 1
        
    # Works on m x b matrices of m input channels and b different inputs
    def forward(self, A):# A is m x K: m input channels and mini-batch size K
        # Store last inputs and K for next backward() call
        self.A = A
        self.K = A.shape[1]
        
        self.mus = np.mean(A, axis=1, keepdims=True)
        self.vars = np.var(A, axis=1, keepdims=True)

        # Normalize inputs using their mean and standard deviation
        self.norm = (A - self.mus) / (np.sqrt(self.vars) + self.eps)
            
        # Return scaled and shifted versions of self.norm
        return self.G * self.norm + self.B

    def backward(self, dLdZ):
        # Re-usable constants
        std_inv = 1/np.sqrt(self.vars+self.eps)
        A_min_mu = self.A-self.mus
        
        dLdnorm = dLdZ * self.G
        dLdVar = np.sum(dLdnorm * A_min_mu * -0.5 * std_inv**3, axis=1, keepdims=True)
        dLdMu = np.sum(dLdnorm*(-std_inv), axis=1, keepdims=True) + dLdVar * (-2/self.K) * np.sum(A_min_mu, axis=1, keepdims=True)
        dLdX = (dLdnorm * std_inv) + (dLdVar * (2/self.K) * A_min_mu) + (dLdMu/self.K)
        
        self.dLdB = np.sum(dLdZ, axis=1, keepdims=True)
        self.dLdG = np.sum(dLdZ * self.norm, axis=1, keepdims=True)
        return dLdX

    def sgd_step(self, lrate):
        self.B = self.B - lrate*self.dLdB
        self.G = self.G - lrate*self.dLdG

# 2) Weight sharing (OPTIONAL)

** Note: You can click the arrow on the left of this text block to collapse/expand this optional section and all its code blocks **

In the lab we designed a CNN that can count the number of objects in 1 dimensional images, where each black pixel is represented by a value of 0 and each white pixel is represented by a value of 1. Recall that an object is a consecutive sequence of black pixels ($0$'s). For example, the sequence $0100110$ contains three objects. 

In this problem we want to see how hard/easy it is to train such a network from data.


Our network architecture will be as follows:

*    The first layer is convolutional and you will implement it using the PyTorch `torch.nn.Conv1d` function, with a kernel of size 2 and stride of 1, followed by a ReLu activation (`torch.nn.ReLU`).

*    The second layer is a fully connected `torch.nn.Linear` layer which has a scalar output.

Here is sample usage of the `Conv1d` and `Linear` layers. 

`layer1=torch.nn.Conv1d(in_channels=?, out_channels=?, kernel_size=?,stride=?,padding=?,bias=True)`

Here, `in_channels` is the number of channels in your data (so for example, RGB images have 3 channels). You can think of the `out_channels` variable as the number of filters you are using.

`layer3 = torch.nn.Linear(in_units=?, out_units=?)`

You need to fill in the parameters marked with `?` based on the problem specifications. Note also that in PyTorch, depending on your implementation, you may be forced to use *three* (four if we count ReLU) layers to implement such a network, where one intermediary `Flatten` layer is used to flatten the output of the convolutional layer, before being passed to the dense layer.

Refer to the <a href="https://pytorch.org/docs/stable/nn.html#conv1d">Conv1D</a>, <a href="https://pytorch.org/docs/stable/nn.html#linear">Linear</a> and <a href="https://pytorch.org/docs/stable/nn.html#flatten">Flatten</a> descriptions in the PyTorch documentation to see the available parameter options.

In this exercise, we fix the structure and want to learn the best combination of weights from data. In the homework code, we have provided functions `train_neural_counter` and `get_image_data_1d`. You can use them to generate data and train the above neural network in PyTorch to answer the following questions. We assume that the images in our data set are randomly generated. The probability of a pixel being white is $0.1$. We work with mean squared error as the loss function for this problem. We have provided template code which you can fill in, to perform the training.

We have also provided helper functions such as `set_weight`, `set_bias`, which might help you set weights and biases of a particular layer.

<b>2B)</b> What is (approximately) the expected loss of the network on $1024\times 1$ images if the convolutional layer is an averaging filter and second layer is the sum function (without a bias term)? (Note that you can answer the question theoretically or through coding, depending on your preference.)

In [0]:
# Code template if you would like to check 2B) through code
tsize = 1000
imsize = 1024
prob_white = 0.1

(X_train,Y_train,X_val,Y_val,X_test,Y_test) = get_image_data_1d(tsize,imsize,prob_white)
test_loader = make_iter(X_test, Y_test)

num_filters = 1
kernel_size = 2
strides = 1
padding = 1

layer_1 = nn.Conv1d(
    in_channels=1,
    out_channels=num_filters,
    kernel_size=kernel_size,
    stride=strides,
    padding=padding,
    bias=False
)

num_units = imsize+1
layer_3 = nn.Linear(num_units, 1, bias=False)
layers = [layer_1, nn.ReLU(), nn.Flatten(), layer_3]
model = nn.Sequential(*layers)

set_weights(model[0], np.array([1/2,1/2]))
set_weights(model[-1], np.ones(num_units))

model_evaluate(model, test_loader, MSELoss())

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


(100.567, 100.567)

<b>2C)</b> Now suppose we add a bias term of $-10$ to the last layer. What is (approximately) the expected quadratic loss? (Note that you can answer the question theoretically or through coding, depending on your preference.)

In [0]:
# Edit code from 2B) with the bias
bias = -10
set_bias(model[-1], bias)
model_evaluate(model, test_loader, MSELoss())

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


(12.507, 12.507)

<b>2D)</b> Averaging type filters are abundant and form a nearly flat valley of local minima for this problem. It is difficult for the network to find alternative solutions on its own. We need to force our way out of these bad minima and towards a better solution, i.e., an edge detector. To force the first layer to behave as an edge detector, we need to choose a proper **kernel regularizer**. Consider the following functions

$f_1=\sum_i |w_i|$, $f_2=\sum_i |w_i^2|$, $f_3=|\sum_{i} w_i|$. Which one of the choices is likely to guide the network to find an edge detector at the convolution layer?


<a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2020_Spring/courseware/Week8/week8_homework/">Refer to HW8 on MITx.</a>

Implement your choice of regularizers from above in the code (complete the function `filter_reg`). Do not allow any bias in the layers for the rest of the problem. The code generates some random test and training data sets and trains the model on these data. Run a few learning trials (5 or more) for each data set and answer the following questions based on the performance of your model.

**IMPORTANT**: When implementing `filter_reg`, you should use the torch backend operations, imported as "torch" in the code. So for example, `torch.sum` and `torch.abs`, rather than `np.sum` and `np.abs`. This is because the `weights` argument is NOT a numpy object, but rather an internal torch object!

In [0]:
# Implement filter_reg
def filter_reg(weights, lam=1000):
    # We scale the output of the filter by lam
    filter_result = torch.abs(torch.sum(weights))
    return lam * filter_result

In [0]:
def model_reg(model, lam=1000):
    # Don't edit this function!
    filter_weights = model[0].weight
    return filter_reg(filter_weights, lam)

def train_neural_counter(layers, data, regularize=False, lam=1000, display=False):
    (X_train, Y_train, X_val, Y_val, X_test, Y_test) = data
    epochs = 10
    batch = 1

    train_iter, val_iter, test_iter = (make_iter(X_train, Y_train),
                                       make_iter(X_val,Y_val),
                                       make_iter(X_test,Y_test))
    model = nn.Sequential(*layers)
    optimizer = Adam(model.parameters())
    criterion = MSELoss()

    if regularize:
        regularizer = lambda model: model_reg(model, lam)
    else:
        regularizer = None

    model_fit(model, train_iter, epochs, optimizer, criterion, val_iter, 
              history=None,verbose=True, model_reg=regularizer)
    err = model_evaluate(model, test_iter, criterion)
    ws = model[-1].weight
    if display:
        plt.plot(ws)
        plt.show()
    return model,err

<b>2E)</b> For $1024\times 1$ images and training set of size $1000$, is the network **without any regularization** likely to find models that have a mean square error lower than 8 on the test data?

In [0]:
imsize = 1024
prob_white = 0.1

data=get_image_data_1d(1000, imsize, prob_white)

def run_trials(trials=5, lam=None):
    for trial in range(trials):
        num_filters = 1
        kernel_size = 2
        strides = 1
        padding = 1

        layer_1 = nn.Conv1d(
            in_channels=1,
            out_channels=num_filters,
            kernel_size=kernel_size,
            stride=strides,
            padding=padding,
            bias=False
        )

        num_units = imsize + 1
        layer_3 = nn.Linear(num_units, 1, bias=False)
        layers = [layer_1, nn.ReLU(), nn.Flatten(), layer_3]
        model, err=train_neural_counter(layers, data, regularize=lam is not None, lam=lam)
        print(model[0].weight)
        print(torch.mean(model[-1].weight))
        print(err)

In [0]:
run_trials()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch: 0 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 1 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 2 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 3 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 4 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 5 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 6 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 7 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 8 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 9 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
Parameter containing:
tensor([[[-0.6691, -0.1608]]], requires_grad=True)
tensor(1.2457e-07, grad_fn=<MeanBackward0>)
(8715.09, 8715.09)
epoch: 0 | TRAIN: loss 3693.79556 acc 3693.79556 | VALI: loss 31

#### For parts F) to J), simply edit your code from E) with the necessary changes.

<b>2F)</b> Repeat the same experiment, but now with the regularizer you implemented. Try different regularization parameters. Which choice of regularization parameter gives the best prediction results?

In [0]:
for lam in [0, 1, 10, 1000]:
    run_trials(trials=3, lam=lam)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch: 0 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 1 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 2 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 3 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 4 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 5 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 6 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 7 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 8 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
epoch: 9 | TRAIN: loss 8679.942 acc 8679.942 | VALI: loss 8777.495 acc 8777.495
Parameter containing:
tensor([[[-0.0079, -0.1442]]], requires_grad=True)
tensor(0.0001, grad_fn=<MeanBackward0>)
(8715.09, 8715.09)
epoch: 0 | TRAIN: loss 4136.56289 acc 4136.56289 | VALI: loss 79.945

<b>2G)</b> With the above choice of regularization parameter, what is the mean square error of the best network that you find on the test data? Try a few trials (5 or more) for each data test and report the value of the best network. 



#### We expect the training to be easier when there are fewer parameters to learn. Consider images of size $128\times 1$ for the rest of the problem.

<b>2H)</b> Instead of resorting to regularization again, we may instead find a way to reduce the number of parameters. What additional layer can you add to the output of the convolution layer to reduce the number of parameters to be learned without losing any relevant information?

<a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2020_Spring/courseware/Week8/week8_homework/">Refer to HW8 on MITx.</a>

<b>2I)</b> Add the layer you suggested above to your network and run some tests with data sets of size 1000 on $128\times 1$ images.  How many parameters are left to learn with the new structure?

You can find the appropriate documentations for the new types of modules mentioned in the previous problem here:

<a href="https://pytorch.org/docs/stable/nn.html#dropout">Dropout</a>

<a href="https://pytorch.org/docs/stable/nn.html#maxpool1d">MaxPool1d</a>

# 3) MNIST (Digit Classification)

In this section, we'll be looking at the MNIST data set seen already in problem 2. This time, we look at the *complete* MNIST problem where our networks will take an image of *any* digit from $0-9$ as input (recall that problem 2 only looked at digits $0$ and $1$) and try to predict that digit. Note that in general, an image is described as a two-dimensional array of pixels. Here, the image is a <a href="https://en.wikipedia.org/wiki/Grayscale">grayscale</a> image, so each pixel is represented by only one integer value, in the range $0$ to $255$ (compared to RGB images where each pixel is represented by three integer values, encoding intensity levels in red, green, and blue color channels).

Also, we will now use out-of-the-box neural network implementations using PyTorch. State-of-the-art systems have error rates of less than 0.5% percent on this data set (see <a href="http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#4d4e495354">this list</a>).  We'll be happy with an error rate less than 2% since we don't have all year...

You can access the MNIST data for this problem using:
<br><code>train, validation = get_MNIST_data()</code>

In [0]:
def shifted(X, shift):
    n = X.shape[0]
    m = X.shape[1]
    size = m + shift
    X_sh = np.zeros((n, size, size))
    plt.ion()
    for i in range(n):
        sh1 = np.random.randint(shift)
        sh2 = np.random.randint(shift)
        X_sh[i, sh1:sh1+m, sh2:sh2+m] = X[i, :, :]
        # If you want to see the shifts, uncomment
        #plt.figure(1); plt.imshow(X[i])
        #plt.figure(2); plt.imshow(X_sh[i])
        #plt.show()
        #input('Go?')
    return X_sh
  
def get_MNIST_data(shift=0):
    train = MNIST(root='./mnist_data', train=True, download=True, transform=None)
    val = MNIST(root='./mnist_data', train=False, download=True, transform=None)
    (X_train, y1), (X_val, y2) = (train.data.numpy(), train.targets.numpy()), \
                                  (val.data.numpy(), val.targets.numpy())
    if shift:
        X_train = shifted(X_train, shift)
        X_val = shifted(X_val, shift)
    return (X_train, y1), (X_val, y2)

You can run the fully connected MNIST model, using:
<br><code>run_pytorch_fc_mnist(train, validation, layers, epochs)</code>

And, you can run the CNN MNIST test, using:
<br><code>run_pytorch_cnn_mnist(train, validation, layers, epochs)</code>

You will need to design your own `layers` to feed to `run_pytorch_fc_mnist` and `run_pytorch_cnn_mnist`, which will be different than the ones specified by `archs()`. For instance, `layers=[Linear(in_features=64, out_features=4)]` defines a single layer with 64 inputs and 4 output units.
Note that the training procedure, uses <a href="https://pytorch.org/docs/stable/nn.html#crossentropyloss">PyTorch's CrossEntropyLoss</a>, which handles the softmax activations for you, so adding a softmax layer to the end of your network is not necessary and will produce undesired results.
Also, we advise you to use the option `verbose=True` when unsure about the progress made during training of your models.
#### **IMPORTANT:** For this and subsequent questions, use the PyTorch implementation of modules. For example, for a linear layer, use <code>nn.Linear(...)</code>.

<b> 3A)</b> Look at the code and indicate what the difference is between <code>run_pytorch_fc_mnist</code> and <code>run_pytorch_cnn_mnist</code>.

In [0]:
def make_deterministic():
    torch.manual_seed(10)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(10)


def weight_reset(l):
    if isinstance(l, Conv2d) or isinstance(l, Linear):
        l.reset_parameters()

def run_pytorch_fc_mnist(train, test, layers, epochs, verbose=True, trials=1, deterministic=True):
    '''
    train, test = input data
    layers = list of PyTorch layers, e.g. [Linear(in_features=784, out_features=10)]
    epochs = number of epochs to run the model for each training trial
    trials = number of evaluation trials, resetting weights before each trial
    '''
    if deterministic:
        make_deterministic()
    (X_train, y1), (X_val, y2) = train, test
    # Flatten the images
    m = X_train.shape[1]
    X_train = X_train.reshape((X_train.shape[0], m * m))
    X_val = X_val.reshape((X_val.shape[0], m * m))

    val_acc, test_acc = 0, 0
    for trial in range(trials):
        # Reset the weights
        for l in layers:
            weight_reset(l)
        # Make Dataset Iterables
        train_iter, val_iter = make_iter(X_train, y1, batch_size=32), make_iter(X_val, y2, batch_size=32)
        # Run the model
        model, vacc, tacc = \
            run_pytorch(train_iter, val_iter, None, layers, epochs, verbose=verbose)
        val_acc += vacc if vacc else 0
        test_acc += tacc if tacc else 0
    if val_acc:
        print("\nAvg. validation accuracy:" + str(val_acc / trials))
    if test_acc:
        print("\nAvg. test accuracy:" + str(test_acc / trials))


def run_pytorch_cnn_mnist(train, test, layers, epochs, verbose=True, trials=1, deterministic=True):
    if deterministic:
        make_deterministic()
    # Load the dataset
    (X_train, y1), (X_val, y2) = train, test
    # Add a final dimension indicating the number of channels (only 1 here)
    m = X_train.shape[1]
    X_train = X_train.reshape((X_train.shape[0], 1, m, m))
    X_val = X_val.reshape((X_val.shape[0], 1, m, m))

    val_acc, test_acc = 0, 0
    for trial in range(trials):
        # Reset the weights
        for l in layers:
            weight_reset(l)
        # Make Dataset Iterables
        train_iter, val_iter = make_iter(X_train, y1, batch_size=32), make_iter(X_val, y2, batch_size=32)
        # Run the model
        model, vacc, tacc = \
            run_pytorch(train_iter, val_iter, None, layers, epochs, verbose=verbose)
        val_acc += vacc if vacc else 0
        test_acc += tacc if tacc else 0
    if val_acc:
        print("\nAvg. validation accuracy:" + str(val_acc / trials))
    if test_acc:
        print("\nAvg. test accuracy:" + str(test_acc / trials))

<b> 3B)</b> Using one epoch of training, what is the accuracy of a network **with no hidden units** (using the <code>run_pytorch_fc_mnist</code> method) on this data?



In [0]:
train, validation = get_MNIST_data()
layers = [nn.Linear(in_features=28*28, out_features=10)]
run_pytorch_fc_mnist(train, validation, layers, 1, verbose=True)

epoch: 0 | TRAIN: loss 0.19019 acc 0.85258 | VALI: loss 0.23529 acc 0.8536


Avg. validation accuracy:0.8536


<b> 3C)</b> Now, linearly scale the data so that the pixel values are between 0 and 1 and repeat your test with the original layer. What is the accuracy now?

In [0]:
train, validation = get_MNIST_data()

# Scale the images
train = train[0] / 255, train[1]
validation = validation[0] / 255, validation[1]

run_pytorch_fc_mnist(train, validation, layers, epochs=1, verbose=True)

epoch: 0 | TRAIN: loss 0.00766 acc 0.9326 | VALI: loss 0.00858 acc 0.924


Avg. validation accuracy:0.924


### Important: <b>Always scale the data like in 3C) for subsequent problems.</b>


<b> 3E)</b> Using this same architecture, what is the accuracy after the 1st, 5th, 10th, and 15th epochs? Note that this colab notebook 0-indexes epoch output. We're looking for the first, fifth, tenth, and fifteenth number outputted by <code>run_pytorch_fc_mnist(train, validation, layers, 15, verbose=True)</code>

In [0]:
run_pytorch_fc_mnist(train, validation, layers, epochs=15, verbose=True)

epoch: 0 | TRAIN: loss 0.00764 acc 0.93283 | VALI: loss 0.00861 acc 0.9242
epoch: 1 | TRAIN: loss 0.00761 acc 0.93343 | VALI: loss 0.00862 acc 0.924
epoch: 2 | TRAIN: loss 0.00759 acc 0.93355 | VALI: loss 0.00864 acc 0.9236
epoch: 3 | TRAIN: loss 0.00757 acc 0.93363 | VALI: loss 0.00866 acc 0.9237
epoch: 4 | TRAIN: loss 0.00755 acc 0.93373 | VALI: loss 0.00867 acc 0.9236
epoch: 5 | TRAIN: loss 0.00753 acc 0.93388 | VALI: loss 0.00869 acc 0.9236
epoch: 6 | TRAIN: loss 0.00751 acc 0.93415 | VALI: loss 0.00871 acc 0.9234
epoch: 7 | TRAIN: loss 0.00749 acc 0.93418 | VALI: loss 0.00872 acc 0.9233
epoch: 8 | TRAIN: loss 0.00748 acc 0.9342 | VALI: loss 0.00874 acc 0.9232
epoch: 9 | TRAIN: loss 0.00746 acc 0.93425 | VALI: loss 0.00876 acc 0.9231
epoch: 10 | TRAIN: loss 0.00745 acc 0.93447 | VALI: loss 0.00877 acc 0.9234
epoch: 11 | TRAIN: loss 0.00743 acc 0.93452 | VALI: loss 0.00879 acc 0.9229
epoch: 12 | TRAIN: loss 0.00742 acc 0.9347 | VALI: loss 0.00881 acc 0.9229
epoch: 13 | TRAIN: loss 0

<b> 3H)</b> Using one epoch of training, try a single hidden layer, followed by a ReLU activation layer before the final output layer, and gradually increase the units; specifically, try (128, 256, 512, 1024) units and observe the results.  What are the accuracies?
To define a ReLU layer in pytorch simply use <code>ReLU()</code>.


In [0]:
for num in [128,256,512,1024]:
    print(f"Using {num} hidden neurons")
    layers = [nn.Linear(in_features=28*28, out_features=num), nn.ReLU(), nn.Linear(in_features=num, out_features=10)]
    run_pytorch_fc_mnist(train, validation, layers, 1, verbose=True)

Using 128 hidden neurons
epoch: 0 | TRAIN: loss 0.0096 acc 0.91493 | VALI: loss 0.00546 acc 0.9478


Avg. validation accuracy:0.9478
Using 256 hidden neurons
epoch: 0 | TRAIN: loss 0.0083 acc 0.92433 | VALI: loss 0.00464 acc 0.953


Avg. validation accuracy:0.953
Using 512 hidden neurons
epoch: 0 | TRAIN: loss 0.00725 acc 0.93253 | VALI: loss 0.00414 acc 0.9581


Avg. validation accuracy:0.9581
Using 1024 hidden neurons
epoch: 0 | TRAIN: loss 0.0065 acc 0.93748 | VALI: loss 0.00387 acc 0.9596


Avg. validation accuracy:0.9596


<b> 3I)</b> Now, try a network with two hidden layers:
<ul>
  <li>A fully connected layer with 512 hidden units
  <li> A ReLU activation layer
  <li> A fully connected layer with 256 hidden units
  <li> A ReLU activation layer
  <li> A fully-connected layer with 10 output units
    
What is the accuracy?


In [0]:
layers = [
    nn.Linear(in_features=28*28, out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=10)
]

run_pytorch_fc_mnist(train, validation, layers, 1, verbose=True)

epoch: 0 | TRAIN: loss 0.00681 acc 0.93478 | VALI: loss 0.00417 acc 0.9579


Avg. validation accuracy:0.9579



<b> 3J)</b> Build a convolutional network with the following structure:

<ul>
<li> A convolutional layer with 32 filters of size 3 × 3
<li> A ReLU activation layer
<li> A max pooling layer with size 2 × 2
<li> A convolutional layer with 64 filters of size 3 × 3
<li> A ReLU activation layer
<li> A max pooling layer with size 2 × 2
<li> A flatten layer (<b>will flatten to size 1600</b>; try to figure out why!)
<li> A fully connected layer with 128 units 
<li> A ReLU activation layer
<li> A dropout layer with drop probability 0.5
<li> A fully-connected layer with 10 output units
</ul>
To define Convolutional and max pooling layers in PyTorch use the following syntax:
<code> c = Conv2d(in_channels=i, out_channels=o, kernel_size=filter_size); m = MaxPool2d(kernel_size=filter_size) </code>, where <code> i </code> and <code> o</code> are integers and <code>filter_size</code> can either be an integer (for square filters) or a tuple (for non-square filters i.e. (2, 3) for 2x3 filter).
Train it on MNIST for one epoch, using <code>run_pytorch_cnn_mnist</code> (this may take a little while).  What is the accuracy on the validation set?



In [0]:
layers = [
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(in_features=1600, out_features=128),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=128, out_features=10)
]

run_pytorch_cnn_mnist(train, validation, layers, 1, verbose=True)

epoch: 0 | TRAIN: loss 0.0077 acc 0.9249 | VALI: loss 0.00212 acc 0.9786


Avg. validation accuracy:0.9786


<b> 3K)</b> Now, let's compare the performance of a fully connected model and a CNN on data where the characters have been shifted randomly so that they are no longer centered.  

You can build such a data set by calling: <code>train_20, validation_20 = get_MNIST_data(shift=20)</code>. Remember to scale it appropriately.

<b>Note that each image is now 48x48, so you will need to change your layer definitions (size after Flatten will be 6400)</b>.
Run your two-hidden-layer FC architecture from above (problem 3I) on this data and then run the CNN architecture from above (problem 3J), both for one epoch. Report your results.


In [0]:
train_20, validation_20 = get_MNIST_data(shift=20)

# Scale the images
train_20 = (train_20[0] / 255, train_20[1])
validation_20 = (validation_20[0] / 255, validation_20[1])

# Fully Connected
layers_fc = [
    nn.Linear(in_features=48*48, out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=10)
]

run_pytorch_fc_mnist(train_20, validation_20, layers_fc, 1, verbose=True)

# CNN
layers_cnn = [
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(in_features=6400, out_features=128),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=128, out_features=10)
]

run_pytorch_cnn_mnist(train_20, validation_20, layers_cnn, 1, verbose=True)

epoch: 0 | TRAIN: loss 0.02367 acc 0.74463 | VALI: loss 0.01344 acc 0.8601


Avg. validation accuracy:0.8601
epoch: 0 | TRAIN: loss 0.0273 acc 0.70153 | VALI: loss 0.00876 acc 0.9247


Avg. validation accuracy:0.9247


# 4. Raining Cats and Dogs

In this problem, we are going to explore how a model trained on a particular dataset behaves in the general population. We will use the following functions (and [generator](https://wiki.python.org/moin/Generators)).

In [0]:
def load_images(directory, imdir='./'):
    imgs = []
    labels = []
    for i in os.listdir(os.path.join(imdir, directory)):
        img = resize(imread(os.path.join(imdir, directory, i)), (224, 224), anti_aliasing=True)
        imgs.append(np.moveaxis(img, 2, 0))
        if 'cat' in i:
            labels.append(0)
        else:
            labels.append(1)
    imgs = np.array(imgs)
    return imgs, labels

def data_gen(train_images, train_labels, batch_size, eval=True):
    all_idxs = np.arange(len(train_labels))
    idxs = np.random.shuffle(all_idxs)
    i = 0
    while i * batch_size + batch_size < train_images.shape[0]:
        samples = train_images[i*batch_size: (i+1)*batch_size]
        sample_labels = train_labels[i*batch_size: (i+1)*batch_size]
        i += 1
        
        yield torch.tensor(samples, dtype=torch.float), torch.tensor(sample_labels, dtype=torch.long)
    if eval:
        samples = train_images[(i)*batch_size:]
        sample_labels = train_labels[(i)*batch_size:]
        yield torch.tensor(samples, dtype=torch.float), torch.tensor(sample_labels, dtype=torch.long)
def postproc_output(out):
  sm = torch.nn.Softmax(dim=1)
  return sm(out).detach().numpy()

**4A)** Write code to evaluate the model on each of the three test sets, following this pseudocode:
<ol>
    <li> Load model 
    <ul>
      <li> <tt> squeezenet_trained_cats_v_dogs.pt</tt> contains the <tt>state_dict</tt> of the model
        <li> You will need to instantiate the model architecture first by running:
          <tt>model = torchvision.models.squeezenet1_1(num_classes=2)</tt>
        <li> Then use <a href="https://pytorch.org/docs/stable/nn.html?highlight=load_state_dict#torch.nn.Module.load_state_dict"><tt> model.load_state_dict</tt> </a>; make sure to use the parameter <tt>map_location=torch.device('cpu')</tt> when you use <tt>torch.load</tt>. It might also be helpful to read about the general workflow of <a href="https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-model-for-inference"> saving and loading trained models in Pytorch </a>
      <li> Make sure that you are in evaluation model by using <tt> model.eval() </tt>
    </ul>
  <li> Load data from <tt>data_path</tt> [Use our function <tt>load_images</tt>]
  <li> For each batch of data [Use our generator <tt>data_gen</tt>; note that the batch size doesn't really matter here except to keep from having to multiply matrices that are too large. For ease of implementation we suggest <tt>batch_size=1</tt>]:
    <ul>
      <li> Pass the batch of <tt>data</tt> through the model using <tt>model(data)</tt>
        <li> Convert the predictions of the model to guesses (after softmax) [use our function <tt>postproc_output</tt>]
      <li> Compare the guesses to the actual <tt>labels</tt>
    </ul>
    <li> Total the accuracy
</ol>

In [0]:
def evaluate_model(model_path, data_path):
    # Load model
    model = torchvision.models.squeezenet1_1(num_classes=2)
    model.load_state_dict(
        torch.load(model_path, map_location=torch.device('cpu')),
    )
    model.eval()

    # Load data
    batch_size = 1
    imgs, labels = load_images(data_path)
    data_generator = data_gen(imgs, labels, batch_size=1)

    correct, total = 0, 0
    
    # Iterate through data, label in data generator
    for img, label in data_generator:
        pred = model(img)
        pred = postproc_output(pred)
        pred = np.argmax(pred, axis=1)
        correct += pred.item() == label.item()
        total +=1
    
    print(f"Correct prediction {correct / float(total)} of the time.")


Calculate the performance on each of the test sets.


In [0]:
model_path = 'squeezenet_trained_cats_v_dogs.pt'
for data_path in ['test1', 'test2', 'test3']:
    print(f"Running inference for test set '{data_path}'.")
    evaluate_model(model_path, data_path)

Running inference for test set 'test1'.
Correct prediction 0.98 of the time.
Running inference for test set 'test2'.
Correct prediction 0.49 of the time.
Running inference for test set 'test3'.
Correct prediction 0.055 of the time.
